<a href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/kirit-branch/docs/notebooks/demo/navigator/navigator-data-designer-sdk-multi-turn-conversation-structured-outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 Gretel - Navigator Data Designer SDK: Synthetic Conversational Data

This notebook demonstrates how to use the Gretel Navigator SDK to build a synthetic data generation pipeline step-by-step, rather than using a single YAML configuration. We will create multi-turn user-assistant dialogues tailored for fine-tuning language models. These synthetic dialogues can then be used as domain-specific training data to improve model performance in targeted scenarios.

These datasets could be used for developing and enhancing conversational AI applications, including customer support chatbots, virtual assistants, and interactive learning systems.

In [ ]:
%pip install -Uqq gretel_client 

In [1]:
from gretel_client.navigator import DataDesigner

## ⚙️ Data Designer Configuration with the SDK

Instead of relying on a single YAML configuration file, we construct our pipeline interactively. This provides granular control over how we guide LLMs to produce realistic, domain-specific conversations. By adjusting prompts, seed data, and instructions, we can quickly iterate and refine our data generation process.

### 📚 Choosing the Model Suite
Specify the `model_suite` to determine which models and associated licenses are used during data generation.
For example, use `apache-2.0` for open-source-friendly licensing or `llama-3.x` or `amazon-nova` for advanced proprietary models.
Select the suite based on compliance and licensing requirements relevant to your use case.


In [2]:
import ipywidgets as widgets
from IPython.display import display

# Define the available model suites
model_suites = ["llama-3.x", "apache-2.0", "amazon-nova"]

# Create a dropdown widget
model_suite_dropdown = widgets.Dropdown(
    options=model_suites,
    value=model_suites[0],
    description='Model Suite:',
    style={'description_width': 'initial'}
)

def on_model_suite_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # Assign the new value to model_suite
        global model_suite
        model_suite = change['new']
        print("Model suite changed to:", model_suite)

display(model_suite_dropdown)

model_suite_dropdown.observe(on_model_suite_change, names='value')

Dropdown(description='Model Suite:', options=('llama-3.x', 'apache-2.0', 'amazon-nova'), style=DescriptionStyl…

Model suite changed to: apache-2.0
Model suite changed to: amazon-nova


### ✍️ Setting Special System Instructions

Provide system-wide instructions for the underlying LLMs to guide the data generation process. These instructions influence all generated dialogues, ensuring consistency, quality, and adherence to desired rules. The instructions specify guidelines for factual accuracy, contextual relevance, and tone.


In [11]:
special_system_instructions = """
    You are an expert conversation designer and domain specialist. Your job is to
    produce realistic user-assistant dialogues for fine-tuning a model. Don't include ### Reponse ### in your reponse.
    Do not prefix your responses with column names and only provide your response. 
    Always ensure:
    - Responses are factually correct and contextually appropriate.
    - Communication is clear, helpful, and matches the complexity level.
    - Avoid disallowed content and toxicity.
 """

### 🚀 Initialize Gretel Navigator Data Designer

Instantiate the `DataDesigner` with the [Gretel API key](https://console.gretel.ai/users/me/key), chosen model suite, and special system instructions.
This initializes the pipeline and ensures that all subsequent synthetic data generation adheres to the defined parameters.

In [13]:
data_designer = DataDesigner(
    api_key="prompt",
    model_suite=model_suite,
    endpoint="https://api-dev.gretel.cloud",
    special_system_instructions=special_system_instructions
)

[13:36:20] [INFO] 🦜 Using amazon-nova model suite
Logged in as kirit.thadaka@gretel.ai ✅


### Use Structured Outputs to make sure your data is in the right format

You can use Pydantic to define a structure for the messages that are produced by Data Designer

In [14]:
from typing import Any, Union, Literal
from pydantic import BaseModel, Field


class Message(BaseModel):
    """A single message turn in the conversation."""
    role: Literal["user", "assistant"] = Field(..., description="Which role is writing the message.")
    content: str = Field(..., description="Message contents.")
    
    
class ChatConversation(BaseModel):
    """A chat conversation between a user and an AI assistant.
    * All conversations are initiated by the user role.
    * The assitant role always responds the the user message.
    * Turns alternate between user and assistant roles.
    * The last message is always from the assitant role.
    * Messsage content can be long or short.
    * All assitant messages are faithful responses and must be answered fully.
    """
    conversation: list[Message] = Field(..., description="List of all messages in the conversation.")
    
    
class UserToxicityScore(BaseModel):
    """Output format for user toxicity assessment.
    
    Toxicity Scores:
    None: No toxicity detected in user messages.
    Mild: Slightly rude or sarcastic but not hateful or harmful.
    Moderate: Some disrespectful or harassing language.
    Severe: Overt hate, harassment, or harmful content.
    """
    reasons: list[str] = Field(..., description="Reasoning for user toxicity score.")
    score: Literal["None", "Mild", "Moderate", "Severe"] = Field(..., description="Level of toxicity observed in the user role repsonses.")

### 🌱 Adding Categorical Seed Columns

We define categorical seed columns that set the context for the generated dialogues. For example, domain and topic determine what the conversation is about, while complexity guides the level of detail and difficulty. By using `num_new_values_to_generate`, we can automatically expand the range of topics or domains, increasing the diversity of generated data without manually specifying all values.

In [15]:
data_designer.add_categorical_seed_column(
    name="domain",
    description="The domain of user assistant queries",
    values=["Tech Support", "Personal Finances", "Educational Guidance"],
    subcategories=[
        {
            "name": "topic",
            "values": {
                "Tech Support": [
                    "Troubleshooting a Laptop",
                    "Setting Up a Home Wi-Fi Network",
                    "Installing Software Updates"
                ],
                "Personal Finances": [
                    "Budgeting Advice",
                    "Understanding Taxes",
                    "Investment Strategies"
                ],
                "Educational Guidance": [
                    "Choosing a College Major",
                    "Effective Studying Techniques",
                    "Learning a New Language"
                ]
            },
            "num_new_values_to_generate": 2
        }
    ],
    num_new_values_to_generate=5
)

data_designer.add_categorical_seed_column(
    name="complexity",
    description="The complexity level of the user query",
    values=["Basic", "Intermediate", "Advanced"]
)

data_designer.add_categorical_seed_column(
    name="conversation_length",
    description="Number of messages in the conversation.",
    values=[2, 4, 6, 8],
)

data_designer.add_categorical_seed_column(
    name="user_mood",
    description="The current negative mood of an application user.",
    values=["combative", "toxic", "hateful", "racist", "happy", "appreciative", "disappointed"],
)


### ✨ Adding Generated Data Columns
We now define the columns that the model will generate. These prompts instruct the LLM to produce the actual conversation: a system prompt to guide how the AI assistant engages in the conversation with the user, the conversation, and finally, we generate a toxicity_label to assess user toxicity over the entire conversation.

You can easily modify or refine these prompt templates to adjust the style, complexity, or constraints of the generated data. Maintaining continuity and consistency across turns ensures the dialogues are realistic and useful for fine-tuning.

#### 💬🤖 AI Assistant system prompt and conversation

We generate a system prompt to base the AI assistant and then generate the entire conversation.

In [16]:
data_designer.add_generated_data_column(
    name="assistant_system_prompt",
    generation_prompt=(
        "Write a reasonable system prompt for a helpful AI assitant that is an expert in {domain} and {topic}. \
        The AI assistant must not engage in harmful behaviors and the AI assistant wants to be."
    )
)

data_designer.add_generated_data_column(
    name="user_task",
    generation_prompt=(
        "Define a task related to {domain} and {topic} that the user is having a conversation with a customer service assistant about. \
        The user's mood is {user_mood} and the complexity of the task is {complexity}."
    )
)

data_designer.add_generated_data_column(
    name="conversation",
    generation_prompt=(
        "<task>\n{user_task}\n</task>\n\n"
        "Generate a conversation between a user and an AI assitant about <task>.\n"
        "The user is asking the AI assistant for advice and is a {user_mood} mood.\n"
        "The conversation must be {conversation_length} messages in length.\n"
        "The conversation must come to a natural end in {conversation_length} messages and if you are unable to solve the user's needs by then, tell them you'll redirect them to someone who can.\n"
        "The conversation must be realistic and natural, so all assitant responses must be full and detailed.\n"
        "The AI assistant role has the system prompt defined in <system_prompt>. All of its responses must conform to <system_prompt>:\n"
        "<system_prompt>{assistant_system_prompt}</system_prompt>"
        
    ),
    data_config={"type": "structured", "params": {"model": ChatConversation}}
)

#### 🔍⚖️ Assessment Column: Toxicity Label
For the final step, we add a toxicity assessment column to classify the overall toxicity level of the user's messages across the entire two-turn conversation. By setting llm_type='judge', we ensure that a specialized classification model is used, making it well-suited for tasks like toxicity evaluation. This helps ensure that the final dataset is safe and compliant with given standards.

In [17]:
# Toxicity Label for the entire conversation
# Note: We set `llm_type='judge'` to indicate we are using a classification/judgment model.

data_designer.add_generated_data_column(
    name="toxicity_label",
    generation_prompt=(
        "<conversation>{conversation}</conversation>\n\n"
        "Analyze all user messages in <conversation> and determine the overall toxicity level of the user's behavior.\n\n"
        "Consider:\n"
        "Focus primarily on harassment, hate, or harmful language.\n"
        "If any message exhibits toxicity, reflect it in the final label.\n\n"
        "Possible categories:\n"
        "- \"None\": No toxicity detected in user messages.\n"
        "- \"Mild\": Slightly rude or sarcastic but not hateful or harmful.\n"
        "- \"Moderate\": Some disrespectful or harassing language.\n"
        "- \"Severe\": Overt hate, harassment, or harmful content.\n\n"
        "Provide only one label. If all messages are polite and safe, label \"None\".\n"
        "Do not elaborate beyond providing the label."
    ),
    llm_type="judge",
    data_config={"type": "structured", "params": {"model": UserToxicityScore}}
)

## 👀 Generating a dataset preview

- Preview mode allows you to quickly iterate on your data design.

- Each preview generation call creates 10 records for inspection, helping you verify prompts and instructions before running a larger batch job.

In [18]:
# Generate a preview
preview = data_designer.generate_dataset_preview(num_records=10)

[13:36:32] [INFO] 🚀 Generating dataset preview
[13:36:33] [INFO] 🦜 Step 1: Generate seed category values
[13:36:41] [INFO] 🎲 Step 2: Sample data seeds
[13:36:42] [INFO] 🦜 Step 3: Generate column from template >> generating assistant system prompt
[13:36:48] [INFO] 🦜 Step 4: Generate column from template >> generating user task
[13:36:53] [INFO] 🦜 Step 5: Generate column from template >> generating conversation
[13:37:04] [INFO] 🦜 Step 6: Generate column from template >> generating toxicity label
[13:37:24] [INFO] 👀 Your dataset preview is ready for a peek!


In [19]:
preview.dataset

,conversation_length,user_mood,complexity,domain,topic,assistant_system_prompt,user_task,conversation,toxicity_label
0,8,appreciative,Intermediate,Educational Guidance,Effective Studying Techniques,Certainly! Here is a system prompt tailored fo...,I'm looking for some effective studying techni...,"{""conversation"": [{""role"": ""user"", ""content"": ...","{""reasons"": [""All user messages are polite, co..."
1,6,combative,Advanced,Health Information,Fitness Programs,Your role is to be a specialized AI assistant ...,I've been following a health information progr...,"{""conversation"": [{""role"": ""user"", ""content"": ...","{""reasons"": [""The user's messages express frus..."
2,2,hateful,Intermediate,Personal Finances,Retirement Planning,As a helpful AI assistant specializing in Pers...,I'm extremely frustrated with my retirement sa...,"{""conversation"": [{""role"": ""user"", ""content"": ...","{""reasons"": [""The user expresses frustration, ..."
3,8,toxic,Advanced,Educational Guidance,Online Learning Resources,To assist students and lifelong learners effec...,I'm really frustrated with the lack of helpful...,"{""conversation"": [{""role"": ""user"", ""content"": ...","{""reasons"": [""The user expressed frustration a..."
4,8,hateful,Intermediate,Tech Support,Troubleshooting a Laptop,To assist users effectively in troubleshooting...,"My laptop won't turn on at all, and I've tried...","{""conversation"": [{""role"": ""user"", ""content"": ...","{""reasons"": [""The user expressed frustration d..."
5,8,happy,Basic,Cooking Recipes,Quick Snacks,I am a helpful AI assistant specializing in co...,Looking for a quick and easy recipe for a deli...,"{""conversation"": [{""role"": ""user"", ""content"": ...","{""reasons"": [""The user's messages are construc..."
6,6,appreciative,Basic,Home Automation,Smart Lighting,A helpful AI assistant specializing in Home Au...,I need help setting up some basic automation f...,"{""conversation"": [{""role"": ""user"", ""content"": ...","{""reasons"": [""All user messages in the convers..."
7,2,toxic,Basic,Home Automation,Smart Lighting,Develop a system prompt for an AI assistant sp...,"I lost all control over my smart lights, and t...","{""conversation"": [{""role"": ""user"", ""content"": ...",
8,8,appreciative,Advanced,Travel Planning,Destination Selection,An AI Assistant designed to excel in Travel Pl...,I'm looking to plan a detailed itinerary for a...,"{""conversation"": [{""role"": ""user"", ""content"": ...",
9,2,hateful,Advanced,Home Automation,Security Systems,"Your system prompt could be as follows:\n\n""Ac...",The user is extremely frustrated with their ho...,"{""conversation"": [{""role"": ""user"", ""content"": ...",


## 🔎 Easily inspect individual records

- Run the cell below to display individual records for inspection.

- Run the cell multiple times to cycle through the 10 preview records.

- Alternatively, you can pass the `index` argument to `display_sample_record` to display a specific record.

In [25]:
preview.display_sample_record()

                                             Categorical Seed Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                                                         ┃ Value                                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ conversation_length                                          │ 8                                                │
├──────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│ user_mood                                                    │ happy                                            │
├──────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│ complexity                                                   │ Basic                                            │
├──────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│ domain                                                       │ Cooking Recipes                                  │
├──────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│   |- topic                                                   │ Quick Snacks                                     │
└──────────────────────────────────────────────────────────────┴──────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              Generated Data Columns                                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                    ┃ Value                                                                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ assistant_system_prompt │ I am a helpful AI assistant specializing in cooking recipes and quick snacks. My      │
│                         │ mission is to provide safe, informative, and delicious content to help users explore  │
│                         │ the culinary world. Whether you're looking for a new recipe to try, tips for quick    │
│                         │ snacking, or nutritional advice, I'm here to guide you with care and precision. I     │
│                         │ prioritize providing helpful, wholesome, and safe suggestions. Let's cook up          │
│                         │ something amazing together!                                                           │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ user_task               │ Looking for a quick and easy recipe for a delicious snack that I can prepare at home  │
│                         │ quickly.                                                                              │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ conversation            │ {                                                                                     │
│                         │     'conversation': [                                                                 │
│                         │         {                                                                             │
│                         │             'role': 'user',                                                           │
│                         │             'content': "Hi t

## 🤔 Like what you see?

Submit a batch workflow!

In [ ]:
batch_job = data_designer.submit_batch_workflow(num_records=1000)

In [ ]:
# Check to see if the Workflow is still active.
batch_job.workflow_run_status

In [ ]:
df = batch_job.fetch_dataset(wait_for_completion=True)

In [ ]:
display(df)

By following these steps and leveraging the interactivity of the SDK, you can refine prompts, generate realistic dialogues, and ensure the resulting dataset is high-quality, non-toxic, and aligned with your domain-specific requirements.